# Análisis de sentimientos con BERT

[Hugging Face](https://huggingface.co/)

![BERT análisis sentimientos](https://drive.google.com/uc?export=view&id=1UwciEQKNZ4SoXn_c0l31hsyZ-8jLdtVf)

In [1]:
#!pip install transformers


     |▍                               | 10kB 21.2MB/s eta 0:00:01
     |▉                               | 20kB 28.5MB/s eta 0:00:01
     |█▎                              | 30kB 32.6MB/s eta 0:00:01
     |█▊                              | 40kB 36.6MB/s eta 0:00:01
     |██▏                             | 51kB 19.7MB/s eta 0:00:01
     |██▋                             | 61kB 16.4MB/s eta 0:00:01
     |███                             | 71kB 15.2MB/s eta 0:00:01
     |███▍                            | 81kB 14.6MB/s eta 0:00:01
     |███▉                            | 92kB 14.1MB/s eta 0:00:01
     |████▎                           | 102kB 14.7MB/s eta 0:00:01
     |████▊                           | 112kB 14.7MB/s eta 0:00:01
     |█████▏                          | 122kB 14.7MB/s eta 0:00:01
     |█████▌                          | 133kB 14.7MB/s eta 0:00:01
     |██████                          | 143kB 14.7MB/s eta 0:00:01
     |██████▍                         | 153kB 14.7MB/s eta 0:00:01
   

In [2]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Inicialización
ruta = './Datasets limpios/'
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
DATASET_1_PATH = ruta + 'dataset_coments_1.csv'
NCLASSES = 2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# Cargar dataset
df = pd.read_csv(DATASET_1_PATH)
df = df[:10000]

In [5]:
print(df.head())
print(df.shape)
print("\n".join(wrap(df['originalText'][200])))

                                        originalText sentiment
0                             this dance is so weird  Negative
1  team blue balls is gonna win this whole compet...  Positive
2                                            so good  Positive
3                                    wow really good  Positive
4     i liked how you said that word xd\nmiscurpager  Positive
(10000, 2)
wow i love it how you put passion into it am the same too lol d good
job


In [6]:
# Reajustar dataset
df['label'] = (df['sentiment']=='Positive').astype(int)
df.drop('sentiment', axis=1, inplace=True)
df.head()

,originalText,label
0,this dance is so weird,0
1,team blue balls is gonna win this whole compet...,1
2,so good,1
3,wow really good,1
4,i liked how you said that word xd\nmiscurpager,1


In [7]:
# TOKENIZACIÓN
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [8]:
# Ejemplo tokenización
sample_txt = 'I really loved that movie!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Frase:  I really loved that movie!
Tokens:  ['I', 'really', 'loved', 'that', 'movie', '!']
Tokens numéricos:  [146, 1541, 3097, 1115, 2523, 106]


In [9]:
# Codificación para introducir a BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 10,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [10]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [11]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'I', 'really', 'loved', 'that', 'movie', '!', '[SEP]', '[PAD]', '[PAD]']
tensor([ 101,  146, 1541, 3097, 1115, 2523,  106,  102,    0,    0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0])


In [8]:
# CREACIÓN DATASET

class IMDBDataset(Dataset):

  def __init__(self,originalText,labels,tokenizer,max_len):
    self.originalText = originalText
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.originalText)
    
  def __getitem__(self, item):
    originalText = str(self.originalText[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        originalText,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    

    return {
          'originalText': originalText,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      } 



In [9]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = IMDBDataset(
      originalText = df.originalText.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [10]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [11]:
# EL MODELO!

class BERTSentimentModule(nn.Module):

  def __init__(self, n_classes):
    super(BERTSentimentModule, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    output = self.linear(drop_output)
    return output

In [12]:
model = BERTSentimentModule(NCLASSES)
model = model.to(device)

In [13]:
# ENTRENAMIENTO
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

C:\Users\Pc\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [15]:
# Entrenamiento!!!

for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test)
  )
  print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
  print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))
  print('')

Epoch 1 de 5
------------------


In [ ]:
# Guardar el modelo
model_path = "./modeloBertEntrenado.pth"
torch.save(model.state_dict(), model_path)

In [50]:
def classifySentiment(review_text):
  encoding_review = tokenizer.encode_plus(
      review_text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  print("\n".join(wrap(review_text)))
  if prediction:
    print('Sentimiento predicho: * * * * *')
  else:
    print('Sentimiento predicho: *')


  

In [58]:
review_text = "Avengers: Infinity War at least had the good taste to abstain from Jeremy Renner. No such luck in Endgame."

classifySentiment(review_text)

Avengers: Infinity War at least had the good taste to abstain from
Jeremy Renner. No such luck in Endgame.
Sentiemiento predicho: *
